In [62]:
import torch
import gymnasium as gym
import numpy as np
import random

In [140]:
env = gym.make("CartPole-v1",render_mode = "human")


In [66]:
class DQN(torch.nn.Module):
    def __init__(self,nb_action, nb_observation):
        super().__init__()
        self.stack = torch.nn.Sequential(
            torch.nn.Linear(nb_observation,32,dtype=float),
            torch.nn.ReLU(),
            torch.nn.Linear(32,16,dtype= float),
            torch.nn.ReLU(),
            torch.nn.Linear(16,nb_action,dtype=float)
        )
    def forward(self,x):
        return self.stack(x)

In [128]:
epsilon = 0.01
def policiy(q_values: torch.Tensor):
    rand = random.random()
    if(rand < epsilon):
        return random.randint(0,q_values.size(0)-1)
    return q_values.argmax(0).item()


In [68]:
dqn = DQN(2,4)

DQN

In [142]:
y = 0.9

lossFn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(dqn.parameters())

def train():
    observation, info = env.reset()
    is_done =False
    total_reward = 0
    while not is_done:
        q_values = dqn(torch.tensor(observation,dtype=float))
        action = policiy(q_values)

        next_observation, reward, terminated, truncated, info = env.step(action) 
        reward -=abs(observation[0])
        total_reward += reward
        with torch.no_grad():
            next_q_values = dqn(torch.tensor(next_observation, dtype=float))  # Target network Q-values
            max_next_q = next_q_values.max(0)[0]  # Max Q-value for the next state
            target_q = reward + y * max_next_q * (1 - terminated)  # Bellman update


        q_value = q_values[action]
        loss = lossFn(q_value, target_q)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


        observation = next_observation
        is_done = truncated or terminated
    print(total_reward)

for i in range(0,1000):
    train()

63.924335


KeyboardInterrupt: 